## Data Preprocessing

The purpose of this notebook is to record the steps and thought process for preprocessing the NIS data for the purposes of developing an autoencoder.

### Data Acquisition

1. Since the NIS switched to ICD-10 starting in 2016, and because the sampling was different (e.g. 1/5 of all hospitals sampled until 2012, when 1/5 of all patients were sampled instead) before 2012, for the purposes of consistency, we'll only use data from **2012 to 2015**.

2. We'll be using Sean's compiled NIS_<year>_Total datasets, which combine the NIS_<year>_Core and NIS_<year>_Severity files together. Outside the scope of this notebook, the SAS7BDAT dataset files will be converted into CSVs, so that in Python, they may be converted into a more efficient HDF5 format.

### Data Schema

Let's load in the data and get information on all present variables so we can create schema for the HDF5 dataset.

In [1]:
import numpy as np
import pandas as pd
import tables

In [2]:
import os
import sys
import re

In [3]:
DATA_FOLDER = '../data/raw/'
MISS_VAL_FILL = -128
MISSING_VALS = ['A', 'B', 'C', 'M', 'F', 'B', 'invl', 'incn', MISS_VAL_FILL]

First, we'll load in the 2012 dataset to get an idea of what we're working with. Major caveat; we'll use chunking to be memory friendly.

In [6]:
nis_csv = pd.read_csv(DATA_FOLDER + 'NIS_2015_TOTAL.csv', chunksize=50000, low_memory=False)
chunk = nis_csv.get_chunk()

Let's see what each column's datatype is:

In [7]:
for col in chunk.columns:
    print("{0} :::: {1}".format(col, chunk[col].dtype))

AGE :::: object
AGE_NEONATE :::: float64
AMONTH :::: int64
AWEEKEND :::: int64
DIED :::: float64
DISCWT :::: float64
DISPUNIFORM :::: float64
DQTR :::: int64
ELECTIVE :::: float64
FEMALE :::: object
HCUP_ED :::: int64
HOSP_DIVISION :::: int64
HOSP_NIS :::: int64
KEY_NIS :::: int64
LOS :::: int64
NIS_STRATUM :::: int64
PAY1 :::: object
PL_NCHS :::: float64
RACE :::: float64
TOTCHG :::: object
TRAN_IN :::: float64
TRAN_OUT :::: float64
YEAR :::: int64
ZIPINC_QRTL :::: object
APRDRG :::: int64
APRDRG_Risk_Mortality :::: int64
APRDRG_Severity :::: int64
CM_AIDS :::: float64
CM_ALCOHOL :::: float64
CM_ANEMDEF :::: float64
CM_ARTH :::: float64
CM_BLDLOSS :::: float64
CM_CHF :::: float64
CM_CHRNLUNG :::: float64
CM_COAG :::: float64
CM_DEPRESS :::: float64
CM_DM :::: float64
CM_DMCX :::: float64
CM_DRUG :::: float64
CM_HTN_C :::: float64
CM_HYPOTHY :::: float64
CM_LIVER :::: float64
CM_LYMPH :::: float64
CM_LYTES :::: float64
CM_METS :::: float64
CM_NEURO :::: float64
CM_OBESE :::: float64
CM

What have we found?

Actually, let's convert all uint32's to int32s, which would allow us to encode missing values as -126 instead of NaN, since the PRDAYn field permits procedures performed before hospitalization to be recorded (min = -4).

| Variable | Description | Loaded Type | Expected Type | Mapping | Mapped Type |
| --- | --- | --- | --- | --- | --- |
| AGE | Patient age | object | uint8 | 
| AGE_NEONATE| Neonatal age | float64 | int8 | 
| AMONTH | Admission month | int32 | int8 |
| AWEEKEND | Admission day is on a weekend | int32 | int8 |
| BODYSYSTEMn | ICD9CM body system n | float64 | int8 |
| CHRONn | ICD-9-CM Chronic Condition Indicators | int32, float64, or object | int8 |
| CHRONBn | Chronic Condition Indicators - body system | int32, float64, or object | int8 |
| CM_<comorbidity>| Presence of a comorbidity | int64 | int8 |
| DIED | Death during hospitalization stay | float64 | int8 |
| DISCWT | Weights | float64 | float64 |
| DISPUNIFORM | disposition | float64 | int8 |
| DQTR | discharge quarter | int32 | int8 |
| DRG | DRG in use on discharge date | int32 | int32 |
| DRG24 | DRG, Version 24 | int32 | int32 |
| DRGVER | DRG or MS-DRG grouper version used on discharge date | int32 | int32 |
| DRG_NoPOA | DRG in use on discharge date, calculated without POA | int32 | int32 |
| DXn| ICD-9-CM Diagnosis code | int32, float64, or object | int32 | map V, E to ints | int32 |
| DXCCSn| Clinical Classifications Software (CCS): ICD-9-CM diagnosis classification | int32, float64, or object | int32 |
| DXMCCS1 | Multi-Level CCS for ICD-9-CM Diagnoses | object | int32 | map V, E to ints | int32 |
| E_MCCS1 | Multi-Level CCS for ICD-9-CM External Cause of Injury Code	| object | int32 | map V, E to ints | int32 |
| ECODEn | ICD-9-CM External Cause of Injury Code | object | int32 | map EXXXX to XXXX, remove invl's |
| ELECTIVE | Elective vs non-elective admission | object | int8 | 
| E_CCS1 | Single-level CCS for ICD-9-CM External Cause of Injury Code | float64 | int32 |
| FEMALE | sex/gender | float64 | int8 | 
| HCUP_ED| HCUP indicator of emergency department record | int32 | int8 |
| HOSP_NIS| Hospital ID | int64 | int64 |
| HOSPBRTH | HCUP indicator indicating in-hospital birth | int32 | int8 |
| HOSP_DIVISION | Census Division of hospital (STRATA) | int32 | int8 |
| KEY_NIS | Admission ID | int64 | int64 |
| LOS | Length of stay | object | int16 |
| MDC | MDC (Major Diagnostic Category) in effect on discharge date	| int64 | int16 | 
| MDC24 | MDC (Major Diagnostic Category) version 24 | int64 | int16 | 
| MDC_NoPOA | MDC (Major Diagnostic Category) in effect on discharge date, calc w/o POA | int64 | int16 | 
| NCHRONIC | Number of chronic conditions | int32 | int8 |
| NDX | Number of ICD-9-CM diagnoses on this discharge | int32 | uint8 |
| NECODE | Number of ICD-9-cM EoCoI Codes on this Record | int32 | uint8 |
| NEOMAT | Neonatal or maternal ICD-9-CM DX / PR | int32 | uint8 |
| NIS_STRATUM | Stratum used to post-stratify hospital | int32 | uint32 |
| NPR | Number of ICD-9-CM procedures on this discharge | int32 | uint8 |
| ORPROC | Major operating room ICD-9-CM procedure indicator | int32 | int8 |
| PAY1 | Expected primary payer | object | int8 |
| PCLASSn | ICD-9-CM Procedure class | float64 | int8 | 
| PL_NCHS2006| Category urban-rural classification scheme | float64 | int8 |
| PRn| Procedure codes |  int32, float64, or object | int32 | map V, E to ints | int32 |
| PRCCSn | CCS class for ICD-9-CM procedures | float64 | int16 |
| PRDAYn| Number of days since admission to procedure n| object, float64 | int16 |
| PRMCCS1 | Multi-Level CCS for ICD-9-CM Procedures	| object | int32 | map V, E to ints | int32 |
| RACE | Race | float64 | int8 |
| TOTCHG | Total charges, cleaned | object | int32 |
| TRAN_IN | Indicator of transfer into the hospital | float64 | int8 |
| TRAN_OUT | Indicator of a transfer outside of the hospital | float64 | int8 |
| YEAR | Year of hospitalization | int64 | int16 |
| ZIPINC_QRTL | Mean household income for zipcode | float64 | int8 |
| APRDRG | All patient refined DRG | int64 | int8 |
| APRDRG_Risk_Mortality | All Patient Refined DRG: Risk of Mortality Subclass | int64 | int8 |
| APRDRG_Severity | All Patient Refined DRG: Severity of Illness Subclass | int64 | int8 |

In [8]:
def is_missing(code):
    return (pd.isna(code) or (code in MISSING_VALS))
    
def map_missing_codes(series):
    """ Replaces missing values with -127 in the series.
    
    Arguments:
        series: 
    
    """
    return series.apply(lambda cell: MISS_VAL_FILL if (pd.isna(cell) or (cell in MISSING_VALS)) else cell)


def map_icd9_codes(series):
    """ Maps alphanumeric ICD-9 codes to purely numeric, notably of type int32.
    
    Wrapper that takes a pandas Series containing ICD-9 codes and maps it based on the following logic:
    000-999 => 0 - 99,999 (should be this way already)
    V00-V99 => 100,000 - 109,999
    E000-E999 => 200,000 - 299,999 (likely 299,99).
    
    Arguments:
        series: Pandas data series containing ICD-9 codes. May be of type object.
        
    Returns:
        series: Pandas data series containing aforementioned mapping, of type int32.
        
    """

    return series.apply(map_icd9_to_numeric)

def map_icd9_to_numeric(code):
    """ 
    """
    if is_missing(code):
        return MISS_VAL_FILL

    code_str = str(code)

    if code_str[0] == 'V':
        len_code = len(code_str[1:])
        scale = 10**(4 - len_code)
        base = 100000
        icd9_nums = int(float(code_str[1:]))

    elif code_str[0] == 'E':
        len_code = len(code_str[1:])
        scale = 10**(5 - len_code)
        base = 200000
        icd9_nums = int(float(code_str[1:]))

    else:
        len_code = len(code_str[0:])
        scale = 10**(5 - len_code)
        base = 0
        icd9_nums = int(float(code_str[0:]))

    code_mapped = base + scale * icd9_nums

    return code_mapped


def map_mccs_codes(series):
    """
    ab.cd.ef.gh => a*10^7 + b*10^6 + c*10^5 + d*10^4 + e*10^3 + f*10^2 + g*10^1 + h*10^0
    """
    
    return series.apply(map_mccs_to_numeric)

def map_mccs_to_numeric(code):
    if is_missing(code):
        return MISS_VAL_FILL
    
    code_str = str(code)
    groups = code_str.split('.')
    code_flat = ''
    for group in groups: 
        code_flat += group

    code_map = 0
    for i, val in enumerate(code_flat):
        scale = 10**(7 - i)
        code_map += int(float(val)) * scale

    return code_map

def map_ecode_to_numeric(code):
    if is_missing(code):
        return MISS_VAL_FILL
    else:
        return int(float(code[1:]))

def map_ecode_codes(series):
    """
    """
    
    return series.apply(map_ecode_to_numeric)

In [9]:
def find_schema(series_name):
    """
    """
    if series_name in SCHEMA_DICT.keys():
        schema_key = series_name
        
    elif series_name[:3] == 'CM_':
        schema_key = 'CM_'
        
    else:
        prefix = re.split('\d', series_name)[0] # Prefix before #
        schema_key = prefix + 'n'
        
    return schema_key, SCHEMA_DICT[schema_key]

In [1]:
SCHEMA_DICT = {'AGE': {'dtype': 'uint8', 'map': []},
'AGE_NEONATE': {'dtype': 'int8', 'map': []},
'AMONTH': {'dtype': 'int8', 'map': []},
'AWEEKEND': {'dtype': 'int8', 'map': []},
'BODYSYSTEMn': {'dtype': 'int8', 'map': []},
'CHRONn': {'dtype': 'int8', 'map': []},
'CHRONBn': {'dtype': 'int8', 'map': []},
'CM_': {'dtype': 'int8', 'map': []},
'DIED': {'dtype': 'int8', 'map': []},
'DISCWT': {'dtype': 'float64', 'map': []},
'DISPUNIFORM': {'dtype': 'int8', 'map': []},
'DQTR': {'dtype': 'int8', 'map': []},
'DRG': {'dtype': 'int32', 'map': []},
'DRG24': {'dtype': 'int32', 'map': []},
'DRGVER': {'dtype': 'int32', 'map': []},
'DRG_NOPOA': {'dtype': 'int32', 'map': []},
'DXn': {'dtype': 'int32', 'map': [map_icd9_codes, ]},
'DXCCSn': {'dtype': 'int32', 'map': []},
'DXMCCSn': {'dtype': 'int32', 'map': [map_mccs_codes, ]},
'DXVER': {'dtype': 'int8', 'map': []},
'E_MCCSn': {'dtype': 'int32', 'map': [map_mccs_codes, ]},
'ECODEn': {'dtype': 'int32', 'map': [map_ecode_codes, ]},
'ELECTIVE': {'dtype': 'int8', 'map': []},
'E_CCSn': {'dtype': 'int32', 'map': []},
'FEMALE': {'dtype': 'int8', 'map': []},
'HCUP_ED': {'dtype': 'int32', 'map': []},
'HOSPBRTH': {'dtype': 'int8', 'map': []},
'HOSP_DIVISION': {'dtype': 'int8', 'map': []},
'HOSP_NIS': {'dtype': 'int64', 'map': []},
'KEY_NIS': {'dtype': 'int64', 'map': []},
'LOS': {'dtype': 'int8', 'map': []},
'MDC': {'dtype': 'int16', 'map': []},
'MDC24': {'dtype': 'int16', 'map': []},
'MDC_NOPOA': {'dtype': 'int16', 'map': []},
'NCHRONIC': {'dtype': 'int8', 'map': []},
'NDX': {'dtype': 'uint8', 'map': []},
'NECODE': {'dtype': 'uint8', 'map': []},
'NEOMAT': {'dtype': 'uint8', 'map': []},
'NIS_STRATUM': {'dtype': 'uint32', 'map': []},
'NPR': {'dtype': 'uint8', 'map': []},
'ORPROC': {'dtype': 'int8', 'map': []},
'PAY1': {'dtype': 'int8', 'map': []},
'PCLASSn': {'dtype': 'int8', 'map': []},
'PL_NCHS2006': {'dtype': 'int8', 'map': []},
'PL_NCHS': {'dtype': 'int8', 'map': []},
'PRn': {'dtype': 'int32', 'map': [map_icd9_codes, ]},
'PRCCSn': {'dtype': 'int16', 'map': []},
'PRDAYn': {'dtype': 'int16', 'map': []},
'PRMCCS1': {'dtype': 'int32', 'map': [map_mccs_codes, ]},
'PRVER': {'dtype': 'int8', 'map': []},
'RACE': {'dtype': 'int8', 'map': []},
'SERVICELINE': {'dtype': 'int8', 'map': []},
'TOTCHG': {'dtype': 'int32', 'map': []},
'TRAN_IN': {'dtype': 'int8', 'map': []},
'TRAN_OUT': {'dtype': 'int8', 'map': []},
'YEAR': {'dtype': 'int16', 'map': []},
'ZIPINC_QRTL': {'dtype': 'int8', 'map': []},
'APRDRG': {'dtype': 'int8', 'map': []},
'APRDRG_RISK_MORTALITY': {'dtype': 'int8', 'map': []},
'APRDRG_SEVERITY': {'dtype': 'int8', 'map': []},}

NameError: name 'map_icd9_codes' is not defined

Okay, so now let's go ahead and start processing this on the chunk level.

In [11]:
for series_name in chunk.columns:
    series_name_caps = series_name.upper()
    schema_key, schema = find_schema(series_name_caps)

In [ ]:
for i, chunk in enumerate(nis_csv):
    for series_name in chunk.columns:
        series_name_caps = series_name.upper()
        schema_key, schema = find_schema(series_name_caps)

        desired_dtype = schema['dtype']

        for mod_fn in schema['map']:
            chunk[series_name] = mod_fn(chunk[series_name])

        chunk[series_name] = map_missing_codes(chunk[series_name])
        chunk[series_name] = chunk[series_name].astype(desired_dtype)
    
    print("Completed Chunk {0}".format(i+1))
    chunk.to_hdf(DATA_FOLDER + 'NIS_2015.h5', "chunk{0}".format(i+1))
    print("Saved Chunk {0}".format(i+1))

### ICD-9 to ICD-10 Mapping

### Combining HDF5 Datasets

Since different datatypes were used to store (maybe consider unifying them all under int32...) the information above, there are actually blocks in each 'chunkXX' group in each HDF5 file, each of which represent all columns pertaining to that data type.

Organization:
File : NIS_201X.h5 => <br>
Group: { chunk_i } => <br>
Group: { axis0, axis1, block0_items, block0_values, ..., block6_values } => <br>
Dataset: { block0_values, ..., block6_values }

In [123]:
import h5py

In [135]:
f['chunk1'].visit(lambda x: print(x, (f['chunk1'][x].shape)))

axis0 (262,)
axis1 (50000,)
block0_items (1,)
block0_values (50000, 1)
block1_items (34,)
block1_values (50000, 34)
block2_items (92,)
block2_values (50000, 92)
block3_items (2,)
block3_values (50000, 2)
block4_items (127,)
block4_values (50000, 127)
block5_items (1,)
block5_values (50000, 1)
block6_items (5,)
block6_values (50000, 5)


First, let's combine all the chunks:

In [444]:
f = h5py.File(DATA_FOLDER + 'NIS_2014.h5', 'r')
fw = h5py.File(DATA_FOLDER + 'NIS_2014_Total.h5', 'a')

# Create group to store all blocks eg datasets
fw_dg = fw.create_group('data')

# Concatenate all chunks
for chunk_id in f.keys():
    chunk = f[chunk_id]
    for key in chunk.keys():
        block = chunk[key]
        
        # Check to see if we have a key established.
        if not key in fw_dg.keys():
            
            if len(block.shape) == 1:
                fw_dg.create_dataset_like(key, block, maxshape=(None,))
                fw_dg[key][:] = block[:]
                
            else:
                rows, cols = block.shape
                fw_dg.create_dataset_like(key, block, shape=(0, cols), maxshape=(None, cols))
            
        if len(block.shape) > 1:
            fw_dg[key].resize((fw_dg[key].shape[0] + block.shape[0]), axis=0)
            fw_dg[key][-block.shape[0]:] = block
            
        else: # Namely for axis0
            if block[0] != fw_dg[key][0]:
                fw_dg[key].resize((fw_dg[key].shape[0] + block.shape[0]), axis=0)
                fw_dg[key][-block.shape[0]:] = block

Now, let's combine all these separate years into a single HDF5 file:

In [491]:
fw_int = h5py.File(DATA_FOLDER + 'NIS.h5', 'a')

for year in ['2012', '2013', '2014']:
    
#     f = h5py.File(DATA_FOLDER + 'NIS_{0}.h5'.format(year), 'r')
    fw = h5py.File(DATA_FOLDER + 'NIS_{0}_Total.h5'.format(year), 'r')

    rows = fw['data']['block0_values'].shape[0]

    fw_int.create_group(year)

    fw_int[year].create_dataset('{0}_data'.format(year), dtype='int32', shape=(rows, 0), maxshape=(rows, None))
    fw_int[year].create_dataset_like('{0}_headers'.format(year), fw['data']['axis0'], shape=(0, ), maxshape=(None, ))

    fw_int[year].create_dataset_like('{0}_ax0'.format(year), fw['data']['axis0'])
    fw_int[year].create_dataset_like('{0}_ax1'.format(year), fw['data']['axis1'])

    fw_int[year]['{0}_ax0'.format(year)][:] = fw['data']['axis0'][:]
    fw_int[year]['{0}_ax1'.format(year)][:] = fw['data']['axis1'][:]

    for block_key in fw['data'].keys():
        if block_key[-3:] == 'ues':
            block = fw['data'][block_key]

            fw_int[year]['{0}_data'.format(year)].resize((fw_int[year]['{0}_data'.format(year)].shape[1] + block.shape[1]), axis=1)
            fw_int[year]['{0}_data'.format(year)][:, -block.shape[1]:] = block[:, :]

            item_key = block_key[:-6] + 'items'
            items = fw['data'][item_key]

            fw_int[year]['{0}_headers'.format(year)].resize(fw_int[year]['{0}_headers'.format(year)].shape[0] + items.shape[0], axis=0)
            fw_int[year]['{0}_headers'.format(year)][-items.shape[0]:] = items[:]

    fw.close()
    f.close()
    
fw_int.close()

In [494]:
fw_int = h5py.File(DATA_FOLDER + 'NIS.h5', 'r')
print(fw_int.keys())

<KeysViewHDF5 ['2012', '2013', '2014']>


Unfortunately, it's unclear whether or not this will read natively, so we may need to construct a handler to manage indexing and searching.

### =======================================================

In [324]:
chunk

<Closed HDF5 group>

In [ ]:
for n, c in chunk['DXCCS1'].value_counts().items(): print(n, c)